<img src="../Pics/MLSb-T.png" width="160">
<br><br>
<center><u><H1>Ensemble Methods</H1></u></center>

In [ ]:
from sklearn.datasets import load_iris
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import LabelEncoder
%matplotlib inline

In [ ]:
iris = load_iris()
X=iris.data[50:,[1, 2]]
y=iris.target[50:]
y

In [ ]:
le = LabelEncoder()
y = le.fit_transform(y)
y

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=1)
sc = StandardScaler()

In [ ]:
X_train_std = sc.fit_transform(X_train)
X_test_std = sc.fit_transform(X_test)

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline

In [ ]:
clf_lr = LogisticRegression(penalty='l2', C=0.001, random_state=0)
clf_dt = DecisionTreeClassifier(max_depth=1, criterion='entropy', random_state=0)
clf_knn = KNeighborsClassifier(n_neighbors=2, p=2, metric='minkowski')

In [ ]:
pipe_lr = Pipeline([['sc', StandardScaler()],['clf_lr', clf_lr]])
pipe_knn = Pipeline([['sc', StandardScaler()],['clf_knn', clf_knn]])

In [ ]:
clf_labels = ['Logistic Regression','Decision Tree', 'KNN']

In [ ]:
for clf, label in zip([clf_lr, clf_dt, clf_knn], clf_labels):
    scores = cross_val_score(estimator=clf,
                            X=X_train_std,
                            y=y_train,
                            cv=10,
                            scoring='roc_auc')
    print('ROC AUC: %0.2f (+/- %0.2f) [%s]' % (scores.mean(), scores.std(), label))

In [ ]:
from sklearn.ensemble import VotingClassifier

In [ ]:
mv = VotingClassifier(estimators=[
    ('lr', pipe_lr), ('dt', clf_dt), ('knn', pipe_knn)], voting='soft')
mv = mv.fit(X_train_std, y_train)
mv_score = cross_val_score(estimator=mv,X=X_train_std,y=y_train,cv=10,scoring='roc_auc')

In [ ]:
print('ROC AUC: %0.2f (+/- %0.2f) [%s]' % (mv_score.mean(), mv_score.std(), 'Mayority Voting'))

## Drawing Classifiers

In [ ]:
res = 0.02
cl_label = ['Logistic Regression','Decision Tree','KNN','Majority Voting']
models = [clf_lr, clf_dt, clf_knn, mv]
x1min, x1max = X_train_std[:,0].min() -1, X_train_std[:,0].max() + 1
x2min, x2max = X_train_std[:,1].min() -1, X_train_std[:,1].max() + 1
xx, yy = np.meshgrid(np.arange(x1min,x1max,res),np.arange(x2min,x2max,res))

for clf, idx, title in zip(models, np.arange(len(models)), cl_label):        
    fig = plt.figure(1,figsize=(10, 10))
    plt.subplot(2, 2, idx + 1)
    clf.fit(X_train_std, y_train)
    output = clf.predict(np.array([xx.ravel(), yy.ravel()]).T)
    output = output.reshape(xx.shape)
    plt.pcolormesh(xx,yy, output, cmap=plt.cm.coolwarm_r)
    plt.scatter(X_train_std[y_train==0, 0], X_train_std[y_train==0, 1], s=30, color='blue', marker='^')
    plt.scatter(X_train_std[y_train==1, 0], X_train_std[y_train==1, 1], s=30, color='yellow', marker='o')
    plt.xlabel(iris.feature_names[1])
    plt.ylabel(iris.feature_names[2])
    plt.title(title)
              
plt.suptitle("Decision surface of different models")
plt.show()

## Reference:
http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.VotingClassifier.html